In [11]:
import ligpy_utils as lig
import constants as const

In [2]:
T = 300

In [3]:
reactionlist, rateconstantlist, compositionlist = lig.set_paths()
y_list = lig.get_specieslist(reactionlist)

In [5]:
speciesindices, indices_to_species = lig.get_speciesindices(y_list)
rate_list = lig.build_rates_list(rateconstantlist, reactionlist, speciesindices, indices_to_species, T)
species_rxns = lig.build_species_rxns_dict(reactionlist)
dydt_expressions = lig.build_dydt_list(rate_list, y_list, species_rxns)
kmatrix = lig.build_k_matrix(rateconstantlist)
kvaluelist = lig.get_k_value_list(T, kmatrix)
PLIGC_0, PLIGH_0, PLIGO_0 = lig.define_initial_composition(compositionlist, 'Pseudotsuga_menziesii')

1.6962168339149761

In [32]:
with open('solve_ODEs_T%s.py' %T, 'w') as f:
    text = "#!/usr/bin/pythons\n" \
            "# -*- coding: utf-8 -*-\n\n" \
            "from scipy.integrate import odeint\n\n\n" \
            "def ODEs(y, t, p):\n"
    f.write(text)
    
    y = '\t' + y_list[0]
    for i in y_list[1:]:
        y += (', ' + i)
    y += ' = y'
    f.write(y + '\n')
    
    p = '\tk_%s_0' %T
    for i in range(1, len(kvaluelist)):
        p += ', k_%s_%s' %(T, i)
    p += ' = p'
    f.write(p + '\n')
    
    dydt = '\tdydt = [%s' %dydt_expressions[0].split('=')[1][1:-2]
    for ODE in dydt_expressions[1:]:
        dydt += (', \n\t\t\t' + ODE.split('=')[1][1:-2])
    dydt += ']'
    f.write(dydt + '\n')
    
    f.write('\treturn dydt\n\n')
    f.write('# k values\n')
    for i in range(len(kvaluelist)):
        f.write('k_%s_%s = %s\n' %(T, i, kvaluelist[i]))
        
    f.write('\n# Initial conditions\n')
    f.write('PLIGC = %s\nPLIGH = %s\nPLIGO = %s\n' %(PLIGC_0, PLIGH_0, PLIGO_0))
    IC = ''
    for i in y_list:
        if i == 'PLIGC' or i == 'PLIGH' or i == 'PLIGO':
            continue
        else:
            IC += ('%s = ' %i)
    IC += '0'
    f.write(IC + '\n\n')
    
    f.write('# ODE solver parameters\n')
    f.write('abserr = %s\n' %const.ABSOLUTE_TOLERANCE)
    f.write('relerr = %s\n' %const.RELATIVE_TOLERANCE)
    f.write('stoptime = 2000\n')
    f.write('numpoints = 10000\n\n')
    
    f.write('t = [stoptime * float(i) / (numpoints - 1) for i in range(numpoints)]\n')
    y0 = 'y0 = [' + y_list[0]
    for spec in y_list[1:]:
        y0 += ', %s' %spec
    y0 += ']'
    f.write(y0)

In [29]:
y0

'y0 = [ADIO, ADIOM2, ALD3, C10H2, C10H2M2, C10H2M4, C2H6, C3H4O, C3H4O2, C3H6, C3H6O2, C3H8O2, CH2CO, CH3CHO, CH3OH, CH4, CHAR, CO, CO2, COUMARYL, ETOH, H2, H2O, KET, KETD, KETDM2, KETM2, LIG, LIGC, LIGH, LIGM2, LIGO, MGUAI, OH, PADIO, PADIOM2, PC2H2, PCH2OH, PCH2P, PCH3, PCHO, PCHOHP, PCHP2, PCOH, PCOHP2, PCOS, PFET3, PFET3M2, PH2, PHENOL, PKETM2, PLIG, PLIGC, PLIGH, PLIGM2, PLIGO, PRADIO, PRADIOM2, PRFET3, PRFET3M2, PRKETM2, PRLIGH, PRLIGH2, PRLIGM2A, RADIO, RADIOM2, RC3H3O, RC3H5O2, RC3H7O2, RCH3, RCH3O, RKET, RKETM2, RLIGA, RLIGB, RLIGH, RLIGM2A, RLIGM2B, RMGUAI, RPHENOL, RPHENOX, RPHENOXM2, SYNAPYL, VADIO, VADIOM2, VCOUMARYL, VKET, VKETD, VKETDM2, VKETM2, VMGUAI, VPHENOL, VSYNAPYL]'

In [ ]:
a

In [ ]:
a

In [ ]:
b

In [ ]:
a = 3

In [ ]:
a

In [ ]:
b

In [ ]:
c